In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
import matplotlib as mpl
import fsspec

In [ ]:
# To save figures
SAVE_PATH = './'
# Results paths
RESULTS_PATH = 'gs://wb2-app-data/v4'  # No trailing slash

## Plotting functions

In [ ]:
# define helper function
def set_y_labels(ax, dataset, levels=True):
    ax.set_xticks([])
    if levels:
      ax.set_yticks(np.arange(len(dataset.level)))
      ax.set_yticklabels(dataset.level.values)
      # ax.tick_params(axis='y', which='major', pad=10)
    else:
      ax.set_yticks([0])
      ax.tick_params(axis='y', color='None')
      ax.set_yticklabels(['000'], color='None')

    ax.spines['top'].set_color('0.7')
    ax.spines['right'].set_color('0.7')
    ax.spines['bottom'].set_color('0.7')
    ax.spines['left'].set_color('0.7')

def add_white_lines(ax, img, color='white'):
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      rect = mpl.patches.Rectangle(
          (j - 0.5, i - 0.5),
          1,
          1,
          linewidth=2,
          edgecolor=color,
          facecolor='None',
      )
      ax.add_patch(rect)

def set_x_labels(ax, dataset, labels=None):
  lead_time_h = int(dataset.lead_time[0] / np.timedelta64(1, 'h'))
  factor_24h = 24 // lead_time_h
  xticks = np.arange(0, len(dataset.lead_time), factor_24h)
  ax.set_xticks(xticks)
  ax.set_xticklabels(xticks + 1 // factor_24h if labels is None else labels)
  ax.spines['top'].set_color('0.7')
  ax.spines['right'].set_color('0.7')
  ax.spines['bottom'].set_color('0.7')
  ax.spines['left'].set_color('0.7')

In [ ]:
plt.rcParams['figure.facecolor'] = '0.9'
# reds = sns.color_palette('Reds', 7)
# blues = sns.color_palette('Blues_r', 7)
# cmap = matplotlib.colors.ListedColormap(blues + [(0.95, 0.95, 0.95)] + reds)
# cb_levels = [-50, -25, -15, -10, -5, -2, -1, 1, 2, 5, 10, 15, 25, 50]
reds = sns.color_palette('Reds', 6)
blues = sns.color_palette('Blues_r', 6)
cmap = matplotlib.colors.ListedColormap(blues + [(0.95, 0.95, 0.95)] + reds)
cb_levels = [-50, -20,-10, -5, -2, -1, 1, 2, 5, 10, 20, 50]
norm = matplotlib.colors.BoundaryNorm(cb_levels, cmap.N, extend='both')
cmap

In [ ]:
def plot_main_scorecard(relative, absolute, variables, save_fn=None,
                        subselect_models=None, nphysical=None, nml=None,
                        custom_titles=None,
                        custom_names=None, custom_subtitles=None):
  matplotlib.rcParams.update(matplotlib.rcParamsDefault)

  models = subselect_models or absolute.model.values

  nmodels = len(models)
  nvariables = len(variables)

  panel_width = 2
  label_width = 1 * panel_width
  padding_right = 0.1

  panel_height = panel_width / 5

  title_height = panel_height * 1.25
  cbar_height = panel_height * 3
  spacing_height = panel_height * 0.1
  spacing_width = panel_height * 0.3

  total_width = label_width + nvariables * panel_width + (nvariables - 1) * spacing_width + padding_right
  total_height = title_height + cbar_height + nmodels * panel_height + (nmodels - 1) * spacing_height

  fig = plt.figure(figsize=(total_width, total_height))
  gs = mpl.gridspec.GridSpec(
      nmodels,
      nvariables,
      figure=fig,
      left=label_width / total_width,
      right=1 - padding_right / total_width,
      top=1-(title_height / total_height),
      bottom=cbar_height / total_height,
      # w/hspace are relative to average panel size
      hspace=spacing_height / panel_height,
      wspace=spacing_width / panel_width
  )
  for row, m in enumerate(models):
    for col, (var, l, metric) in enumerate(variables):
      ax = fig.add_subplot(gs[row, col])

      abs = absolute.sel(model=m, metric=metric)[var]
      da = relative.sel(model=m, metric=metric)[var]
      if l:
        da = da.sel(level=l)
        abs = abs.sel(level=l)
      values = da.sel(lead_time=lead_times).values[None, :]
      abs_values = abs.sel(lead_time=lead_times).values[None, :]
      img = ax.imshow(values, aspect='auto', cmap=cmap, norm=norm)
      set_y_labels(ax, da, levels=False)
      add_white_lines(ax, values, color='0.9')
      ax.grid(False)
      if col == 0:
        if custom_names is None:
          label = m.split(' vs')[0]
        else:
          label = custom_names[row]
        ax.set_ylabel(label, rotation='horizontal', horizontalalignment='right',
                      verticalalignment='center', labelpad=-20, zorder=10, fontsize=9.5)
        # Add physical model box
        if (nphysical is not None) and (row == nphysical-1):
          h=nphysical + (nphysical - 1) * spacing_height / panel_height
          rect = mpl.patches.Rectangle(
              (-0.98, 0), width=0.96, height=h, color="lightsteelblue", transform=ax.transAxes,
              clip_on=False, alpha=0.2, zorder=0.1
          )
          ax.add_patch(rect)
          if nphysical >=3:
            ax.text(-0.9, h/2, 'Physical models', ha='center',va='center',
                    transform=ax.transAxes, fontsize=8, rotation='vertical')
        # Add ML model box
        elif (nml is not None) and (row == nphysical + nml - 1):
          h=nml + (nml - 1) * spacing_height / panel_height
          rect = mpl.patches.Rectangle(
              (-0.98, 0), width=0.96, height=h, color="lightcoral", transform=ax.transAxes,
              clip_on=False, alpha=0.2, zorder=0.1
          )
          ax.add_patch(rect)
          if nml >=3:
            ax.text(-0.9, h/2, 'ML / hybrid models', ha='center',va='center',
                    transform=ax.transAxes, fontsize=8, rotation='vertical')
        else:
          ax.set_zorder(10)
      if row == 0:

        ax.set_title(custom_titles[col], fontsize=10, pad=17)
        ax.text(0.5, 1.25, custom_subtitles[col], ha='center',va='center', transform=ax.transAxes,
                fontsize=6)
      if np.isnan(values).all():
        ax.remove()
        continue


      if row == nmodels - 1:
        lead_time_in_days = absolute.lead_time.values.astype('timedelta64[h]').astype(int) // 24
        set_x_labels(ax, da, labels=lead_time_in_days)
        ax.set_xlabel('Lead time [days]')
      # Add absolute numbers
      for i, v in enumerate(abs_values[0]):
        if m == 'Climatology vs ERA5':
          v = np.mean(abs_values)
        if np.isfinite(v):
          if var == 'Specific Humidity': v *= 1000
          if var == '24h Precipitation' and metric == 'CRPS': v *= 1000

          if v > 10:
            v = str(v)[:3].rstrip('.')
          else:
            v = str(v)[:4]
          ax.text(i, 0, v, ha='center',va='center', fontsize=8)

  rel_cbar_height = cbar_height / total_height
  cax = fig.add_axes((0.35, rel_cbar_height * 0.4, 0.5, rel_cbar_height * 0.1))
  cb = fig.colorbar(img, cax=cax, orientation='horizontal', fraction=0.01)
  cb.ax.set_xticks(cb_levels)
  if '24h Precipitation' in np.ravel(variables):
    cb.ax.set_xlabel(r'Better $\longleftarrow$ % difference in RMSE/SEEPS vs IFS HRES $\longrightarrow$ Worse')
  else:
    cb.ax.set_xlabel(r'Better $\longleftarrow$ % difference in RMSE vs IFS HRES $\longrightarrow$ Worse')

  if save_fn:
    with fsspec.open(f'{SAVE_PATH}/{save_fn}', 'wb', auto_mkdir=True) as f:
      # fig.patch.set_facecolor('gray')
      plt.savefig(f, dpi=300)

      plt.close()

## Data preprocessing functions

In [ ]:
def replace_analysis_with_era_precip(results):
  # Since vs Analysis doesn't have precip scores, replace vs analysis precip scores with ERA precip scores
  precip_variables = ['6h Precipitation', '24h Precipitation']
  for model in results.model.values:
    if model.endswith('vs Analysis'):
      model_vs_era = model.replace('vs Analysis', 'vs ERA5')
      if model_vs_era in results.model.values:
        for v in precip_variables:
          results[v].loc[dict(model=model)] = results[v].sel(dict(model=model_vs_era))
  return results

In [ ]:
def compute_relative_results(results, reference_model):
  # Compute relative result
  reference = results.sel(model=reference_model, drop=True)
  relative = ((results - reference) / reference * 100)
  return relative

## Deterministic

In [ ]:
lead_times = np.array([1, 3, 5, 7, 10], dtype='timedelta64[D]')
models = [
    'IFS HRES vs Analysis',
    'IFS ENS (mean) vs Analysis',
    'ERA5-Forecasts vs ERA5',
    'Pangu-Weather (oper.) vs Analysis',
    'GraphCast (oper.) vs Analysis',
    'GenCast (oper.) (mean) vs Analysis',
    # 'GenCast 100m U/V (oper.) (mean) vs Analysis',
    'Keisler (2022) vs ERA5',
    'Pangu-Weather vs ERA5',
    'GraphCast vs ERA5',
    'FuXi vs ERA5',
    'NeuralGCM 0.7 vs ERA5',
    'NeuralGCM ENS (mean) vs ERA5',
    'GenCast (mean) vs ERA5',
    'Stormer ENS (mean) vs ERA5',
    'Excarta (HEAL-ViT) vs ERA5',
    # 'Swin vs ERA5',
    'ArchesWeather-Mx4 vs ERA5',
    'ArchesWeatherGen (mean) vs ERA5',
    'Climatology vs ERA5'
]

In [ ]:
results = xr.open_zarr(f'{RESULTS_PATH}/deterministic.zarr').compute()

In [ ]:
# Make all coarse selections
results = results.sel(lead_time=lead_times, region='Global', year='2020', resolution='240x121')

In [ ]:
results = replace_analysis_with_era_precip(results)
relative = compute_relative_results(results, 'IFS HRES vs Analysis')

In [ ]:
# Pick models
results = results.sel(model=models)
relative = relative.sel(model=models)

In [ ]:
nphysical = 3
upper_variables = [
    ('Geopotential', 500, 'RMSE'),
    ('Temperature', 850, 'RMSE'),
    ('Specific Humidity', 700, 'RMSE'),
    ('Wind Vector', 850, 'RMSE'),
]
upper_titles = ['Geopotential', 'Temperature', 'Humidity', 'Wind Vector']
upper_subtitles = [f'500hPa geopotential RMSE [kg$^2$/m$^2$]', '850hPa temperature RMSE [K]',
                                     '700hPa specific humidity RMSE [g/kg]', '850hPa wind vector RMSE [m/s]']
surface_variables = [
    ('2m Temperature', None, 'RMSE'),
    ('Sea Level Pressure', None, 'RMSE'),
    ('10m Wind Speed', None, 'RMSE'),
    ('24h Precipitation', None, 'SEEPS'),
]
surface_titles = ['2m Temperature', 'Surface Pressure', '10m Wind Speed', 'Precipitation']
surface_subtitles = ['RMSE [K]', 'RMSE [Pa]',
                                     'RMSE [m/s]', '24h precipitation SEEPS']
surface_models = [
    'IFS HRES vs Analysis',
    'IFS ENS (mean) vs Analysis',
    'ERA5-Forecasts vs ERA5',
    'Pangu-Weather (oper.) vs Analysis',
    'GraphCast (oper.) vs Analysis',
    'GenCast (oper.) (mean) vs Analysis',
    'Pangu-Weather vs ERA5',
    'GraphCast vs ERA5',
    'FuXi vs ERA5',
    'GenCast (mean) vs ERA5',
    'Stormer ENS (mean) vs ERA5',
    'Excarta (HEAL-ViT) vs ERA5',
    # 'Swin vs ERA5',
    'ArchesWeather-Mx4 vs ERA5',
    'ArchesWeatherGen (mean) vs ERA5',
    'Climatology vs ERA5'
]

In [ ]:
plot_main_scorecard(
    relative,
    results,
    variables=upper_variables,
    nphysical=nphysical,
    nml=len(models) - nphysical - 1,
    custom_titles=upper_titles,
    custom_subtitles=upper_subtitles,
    save_fn='deterministic_upper.png'
)
plt.show()

In [ ]:
plot_main_scorecard(
    relative,
    results,
    variables=surface_variables,
    subselect_models=surface_models,
    nphysical=nphysical,
    nml=len(surface_models) - nphysical - 1,
    custom_titles=surface_titles,
    custom_subtitles=surface_subtitles,
    save_fn='deterministic_surface.png'
)
plt.show()

In [ ]:
# from IPython.display import Image
# Image('test.png')

## Probabilistic

In [ ]:
lead_times = np.array([1, 3, 5, 10, 15], dtype='timedelta64[D]')
models = [
    'IFS ENS vs Analysis',
    'GenCast (oper.) vs Analysis',
    'NeuralGCM ENS vs ERA5',
    'GenCast vs ERA5',
    'ArchesWeatherGen vs ERA5',
    'Probabilistic Climatology vs ERA5',
]

In [ ]:
results = xr.open_zarr(f'{RESULTS_PATH}/probabilistic.zarr').compute()

In [ ]:
results

In [ ]:
# Make all coarse selections
results = results.sel(lead_time=lead_times, region='Global', year='2020', resolution='240x121')

In [ ]:
results = replace_analysis_with_era_precip(results)
relative = compute_relative_results(results, 'IFS ENS vs Analysis')

In [ ]:
# Pick models
results = results.sel(model=models)
relative = relative.sel(model=models)

In [ ]:
nphysical = 1
upper_variables = [
    ('Geopotential', 500, 'CRPS'),
    ('Temperature', 850, 'CRPS'),
    ('Specific Humidity', 700, 'CRPS'),
    ('Wind Speed', 850, 'CRPS'),
]
upper_titles = ['Geopotential', 'Temperature', 'Humidity', 'Wind Speed']
upper_subtitles = [f'500hPa geopotential CRPS [kg$^2$/m$^2$]', '850hPa temperature CRPS [K]',
                                     '700hPa specific humidity CRPS [g/kg]', '850hPa wind speed CRPS [m/s]']
surface_variables = [
    ('2m Temperature', None, 'CRPS'),
    ('Sea Level Pressure', None, 'CRPS'),
    ('10m Wind Speed', None, 'CRPS'),
    ('24h Precipitation', None, 'CRPS'),
]
surface_titles = ['2m Temperature', 'Surface Pressure', '10m Wind Speed', 'Precipitation']
surface_subtitles = ['CRPS [K]', 'CRPS [Pa]',
                                     'CRPS [m/s]', '24h precipitation CRPS [mm/h]']
surface_models = [
    'IFS ENS vs Analysis',
    'GenCast (oper.) vs Analysis',
    'GenCast vs ERA5',
    'ArchesWeatherGen vs ERA5',
    'Probabilistic Climatology vs ERA5'
]

In [ ]:
plot_main_scorecard(
    relative,
    results,
    variables=upper_variables,
    nphysical=nphysical,
    nml=len(models) - nphysical - 1,
    custom_titles=upper_titles,
    custom_subtitles=upper_subtitles,
    save_fn='probabilistic_upper.png'
)
plt.show()

In [ ]:
plot_main_scorecard(
    relative,
    results,
    variables=surface_variables,
    subselect_models=surface_models,
    nphysical=nphysical,
    nml=len(surface_models) - nphysical - 1,
    custom_titles=surface_titles,
    custom_subtitles=surface_subtitles,
    save_fn='probabilistic_surface.png'
)
plt.show()